In [5]:
import seaborn as sns
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_formats = ['pdf']

In [6]:
filename = 'grid4rand.json'
with open(filename) as json_file:
    data = json.load(json_file)
testRuns = pd.io.json.json_normalize(data)
testRuns = testRuns.loc[testRuns['tripGenRate'] < 0.05]
testRuns = testRuns.loc[testRuns['tripGenRate'] > 0.00]
testRunsComp = testRuns.loc[testRuns['lastTripStep'] > 3000]
#print(testRuns.loc[0])
testRuns

,step,timeMins,numTravelers,averageTrips,averageSteps,averageStepsPerTrip,tripsCompleted,passengerSteps,stepsWithPassenger,stepsWithoutPassenger,...,stepsTravelingWithoutPassenger,distWithPassenger,distWithoutPassenger,lastTripStep,lanePolicy,numVehicles,grids,density,tripGenRate,seed
0,3600,60.0,105,3800,477524,125,3800,477524,485308,6428612,...,3245687,496626,4876757,3598,True,1920,4,120,0.010141,1578499406404
1,3600,60.0,1557,2262,320423,141,2262,320423,991793,5922127,...,1773152,306896,2920373,2399,False,1920,4,120,0.010141,1578499406404
2,3600,60.0,62,3419,328853,96,3419,328853,332136,4449992,...,2718908,446489,4496111,3599,True,1328,4,83,0.009385,1578499406405
3,3600,60.0,1613,1753,179384,102,1753,179384,773927,4008201,...,1223276,239873,2169666,2042,False,1328,4,83,0.009385,1578499406405
6,3600,60.0,26,1692,146951,86,1692,146951,148253,3654403,...,2490019,220405,4296834,3589,True,1056,4,66,0.004041,1578499406407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,3600,60.0,354,13884,1290753,92,13884,1290753,1308582,2321226,...,1766849,1854103,3216247,3599,False,1008,4,63,0.047226,1578499406462
120,3600,60.0,1898,7474,4162117,556,7474,4162117,5309579,8979189,...,1481570,1058514,1866320,3598,True,3968,4,248,0.025503,1578499406464
121,3600,60.0,753,8723,3244552,371,8723,3244552,3413445,10875323,...,3006646,1175882,3916947,3599,False,3968,4,248,0.025503,1578499406464
122,3600,60.0,310,6175,1434679,232,6175,1434679,1479907,9524749,...,2743312,828660,3478522,3599,True,3056,4,191,0.016853,1578499406465


In [25]:
df = testRuns
numRoadCells = (16*grids*2*(grids*2+1)*2 + (16*grids*2*grids*2)) * 2
df['averageSpeedWithPassenger'] = df['distWithPassenger'].div(df['stepsWithPassenger'])
df['averageSpeedWithoutPassenger'] = df['distWithoutPassenger'].div(df['stepsWithoutPassenger'])
df['averageSpeed'] = (df['distWithPassenger'].add(df['distWithoutPassenger'])).div(df['stepsWithoutPassenger'].add(df['stepsWithPassenger']))
df['averageStepsWithoutPassenger'] = df['stepsWithoutPassenger'].div(df['numVehicles'])
df['averageStepsTravelingWithoutPassenger'] = df['stepsTravelingWithoutPassenger'].div(df['numVehicles'])
df['densityCell'] = df['numVehicles'].div(numRoadCells)
df['densityKm'] = df['numVehicles'].div(numRoadCells)/7.5*1000
densityLabelsCell = np.around(np.arange(0.1, 0.575, 0.025), 3)
densityBinsCell = np.arange(0.075, 0.575, 0.025)
densityLabelsKm = np.arange(20, 75, 5)
densityBinsKm = np.arange(15, 75, 5)
df['densityBin'] = pd.cut(df['densityKm'], densityBinsKm, labels = densityLabelsKm)
testRuns = df
32*grids*grids/numRoadCells/7.5*1000

10.256410256410257

In [12]:
grids = 4
numRoadCells = (16*grids*2*(grids*2+1)*2 + (16*grids*2*grids*2)) * 2
df = testRunsComp.pivot('seed','lanePolicy','averageStepsPerTrip').add_prefix('steps')
df['stepsDiff'] = df['stepsTrue'].sub(df['stepsFalse'])
df['stepsPercent'] = df['stepsDiff'].div(df['stepsFalse'])*100
df['density'] = testRunsComp.drop_duplicates('seed').set_index('seed')['density']
df['tripGenRate'] = testRunsComp.drop_duplicates('seed').set_index('seed')['tripGenRate']
df['numVehicles'] = testRunsComp.drop_duplicates('seed').set_index('seed')['numVehicles']
df['densityCell'] = df['numVehicles'].div(numRoadCells)
df['densityKm'] = df['numVehicles'].div(numRoadCells)/7.5*1000
densityLabelsCell = np.around(np.arange(0.1, 0.575, 0.025), 3)
densityBinsCell = np.arange(0.075, 0.575, 0.025)
densityLabelsKm = np.arange(20, 75, 5)
densityBinsKm = np.arange(15, 75, 5)
df['densityBin'] = pd.cut(df['densityKm'], densityBinsKm, labels = densityLabelsKm)
df = df.reset_index().rename_axis(None, axis=1)
#print (df)
travelTimeDf = df

In [13]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "stepsPercent", data=travelTimeDf,
                facecolor='0.9', edgecolor='0.0')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Travel Time Difference (%)")
#plt.xticks(rotation=-90)
plt.show(g)

<Figure size 360x252 with 1 Axes>

In [14]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeedWithPassenger", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed with Passenger (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

<Figure size 360x252 with 1 Axes>

In [15]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeed", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

<Figure size 360x252 with 1 Axes>

In [16]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeedWithoutPassenger", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed with Passenger (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

<Figure size 360x252 with 1 Axes>

In [17]:
plt.figure(figsize=(5,3.5))
g = sns.scatterplot("density", "averageSpeed", data=testRuns, color='0.4', style='lanePolicy')
#g.set(xlim=(0,50))
#plt.legend(title="Lane Policy")
plt.xlabel("Density")
plt.ylabel("Travel Time (steps)")
plt.show(g)

<Figure size 360x252 with 1 Axes>

In [116]:
plt.figure(figsize=(5,3.5))
g = sns.scatterplot("density", "averageSpeedWithPassenger", data=testRuns, color='0.4', style='lanePolicy')
#g.set(xlim=(0,50))
#plt.legend(title="Lane Policy")
plt.xlabel("Density")
plt.ylabel("Travel Time (steps)")
plt.show(g)

<Figure size 360x252 with 1 Axes>